# Power Outages Analysis

**Name**: Chentao Gong

**Website Link**: [View My Website](https://terantaon.github.io/Power-Outages-Analysis/)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

## Step 1: Introduction

In [2]:
#For this project, I will investigate my research question: What factors affect the cause of major power outages?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
#The column names are on row 5. Row 6 stores the units of each column. The first row of the data set is on row 7.
df = pd.read_excel('data/outage.xlsx', header=5).drop(0)

# There are 57 columns in the original excel file, but only these columns are related to what we want to investigate.
cols = ['YEAR','MONTH','POSTAL.CODE','NERC.REGION','CLIMATE.REGION',
        'ANOMALY.LEVEL','OUTAGE.START.DATE','OUTAGE.START.TIME',
        'OUTAGE.RESTORATION.DATE','OUTAGE.RESTORATION.TIME','CAUSE.CATEGORY',
        'OUTAGE.DURATION','DEMAND.LOSS.MW','CUSTOMERS.AFFECTED','TOTAL.PRICE',
        'TOTAL.SALES','TOTAL.CUSTOMERS','POPULATION','POPPCT_URBAN']
df = df[cols]

#Drop rows with values that are MCAR
row_index = df[df['OUTAGE.START.DATE'].isna()].index
df = df.drop(row_index)
df = df.reset_index().drop('index', axis=1)

#Combine OUTAGE.START.DATE and OUTAGE.START.TIME into one column OUTAGE.START
date = df['OUTAGE.START.DATE'].dropna().astype(str).transform(lambda d : d.split()[0])
time = df['OUTAGE.START.TIME'].dropna().astype(str)
df['OUTAGE.START'] = pd.to_datetime(date + ' ' + time)
#Combine OUTAGE.RESTORATION.DATE and OUTAGE.RESTORATION.TIME into one column OUTAGE.RESTORATION
date = df['OUTAGE.RESTORATION.DATE'].dropna().astype(str).transform(lambda d : d.split()[0])
time = df['OUTAGE.RESTORATION.TIME'].dropna().astype(str)
df['OUTAGE.RESTORATION'] = pd.to_datetime(date + ' ' + time)
df = df.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE','OUTAGE.RESTORATION.TIME'], axis=1)

#Add two features: the hour when outages occur, and whether outages occur at day or night
df['HOUR'] = df['OUTAGE.START'].dt.hour
df['DAY'] = df['HOUR'].between(6, 18, inclusive='left')

#Replace meaningless zeros with NaN
row_idx = df[df['OUTAGE.DURATION'] == 0].index
df.loc[row_idx, 'OUTAGE.DURATION'] = np.nan
df.loc[row_idx, 'OUTAGE.RESTORATION'] = np.nan
row_idx = df[df['CUSTOMERS.AFFECTED'] == 0].index
df.loc[row_idx, 'CUSTOMERS.AFFECTED'] = np.nan
row_idx = df[df['DEMAND.LOSS.MW'] == 0].index
df.loc[row_idx, 'DEMAND.LOSS.MW'] = np.nan

df.head(10)

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,POPULATION,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION,HOUR,DAY
0,2011.0,7.0,MN,MRO,East North Central,-0.3,severe weather,3060,NaN,70000.0,9.28,6562520,2.60e+06,5.35e+06,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00,17,True
1,2014.0,5.0,MN,MRO,East North Central,-0.1,intentional attack,1,NaN,NaN,9.28,5284231,2.64e+06,5.46e+06,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00,18,False
2,2010.0,10.0,MN,MRO,East North Central,-1.5,severe weather,3000,NaN,70000.0,8.15,5222116,2.59e+06,5.31e+06,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00,20,False
3,2012.0,6.0,MN,MRO,East North Central,-0.1,severe weather,2550,NaN,68200.0,9.19,5787064,2.61e+06,5.38e+06,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00,4,False
4,2015.0,7.0,MN,MRO,East North Central,1.2,severe weather,1740,250,250000.0,10.43,5970339,2.67e+06,5.49e+06,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00,2,False
5,2010.0,11.0,MN,MRO,East North Central,-1.4,severe weather,1860,NaN,60000.0,8.28,5374150,2.59e+06,5.31e+06,73.27,2010-11-13 15:00:00,2010-11-14 22:00:00,15,True
6,2010.0,7.0,MN,MRO,East North Central,-0.9,severe weather,2970,NaN,63000.0,9.12,6374935,2.59e+06,5.31e+06,73.27,2010-07-17 20:30:00,2010-07-19 22:00:00,20,False
7,2005.0,6.0,MN,MRO,East North Central,0.2,severe weather,3960,75,300000.0,7.36,5607498,2.47e+06,5.12e+06,73.27,2005-06-08 04:00:00,2005-06-10 22:00:00,4,False
8,2015.0,3.0,MN,MRO,East North Central,0.6,intentional attack,155,20,5941.0,9.03,5599486,2.67e+06,5.49e+06,73.27,2015-03-16 07:31:00,2015-03-16 10:06:00,7,True
9,2013.0,6.0,MN,MRO,East North Central,-0.2,severe weather,3621,NaN,400000.0,10,5490631,2.62e+06,5.42e+06,73.27,2013-06-21 17:39:00,2013-06-24 06:00:00,17,True


In [5]:
month = df[['YEAR', 'MONTH']].groupby('MONTH').count().reset_index().rename(columns={'YEAR' : 'Number of Outages', 'MONTH' : 'Month'})
fig1 = px.bar(month, title='Number of Outages per Month', x='Month', y='Number of Outages')
fig1.write_html('assets/fig1.html', include_plotlyjs='cdn')
fig1

In [518]:
anomaly = df[['YEAR', 'ANOMALY.LEVEL']].groupby('ANOMALY.LEVEL').count().reset_index().rename(columns={'ANOMALY.LEVEL' : 'Anomaly Level', 'YEAR' : 'Number of Outages'})
fig2 = px.line(anomaly, title='Number of Outages Over Anomaly Level', x='Anomaly Level', y='Number of Outages')

In [519]:
temp = df.rename(columns={'CAUSE.CATEGORY' : 'Cause Category', 'CUSTOMERS.AFFECTED' : 'Number of Customers Affected'})
fig3 = px.scatter(temp, title='Customers Affected by Cause Category', x='Cause Category', y='Number of Customers Affected')
fig3

In [520]:
temp = df[['DAY', 'CAUSE.CATEGORY', 'YEAR']].rename(columns={'DAY' : 'Day', 'CAUSE.CATEGORY' : 'Cause Category'})
temp = temp.pivot_table(
    index='Cause Category',
    columns='Day',
    values='YEAR',
    aggfunc='count'
)
fig4 = temp.plot(kind='barh', title='Cause Category by Day/Night', barmode='group')
fig4

In [521]:
df.pivot_table(
    index='CLIMATE.REGION',
    columns='CAUSE.CATEGORY',
    values='YEAR',
    aggfunc='count',
)

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
CLIMATE.REGION,,,,,,,
Central,7.0,4.0,38.0,3.0,2.0,134.0,11.0
East North Central,3.0,5.0,20.0,1.0,2.0,104.0,3.0
Northeast,5.0,14.0,135.0,1.0,4.0,176.0,14.0
Northwest,2.0,1.0,89.0,5.0,2.0,29.0,4.0
South,9.0,7.0,28.0,2.0,42.0,112.0,27.0
Southeast,4.0,NaN,9.0,NaN,5.0,116.0,16.0
Southwest,5.0,2.0,64.0,1.0,1.0,10.0,9.0
West,21.0,17.0,31.0,28.0,9.0,70.0,41.0
West North Central,1.0,NaN,4.0,5.0,2.0,4.0,NaN


In [522]:
df.pivot_table(
    index='NERC.REGION',
    columns='CAUSE.CATEGORY',
    values='YEAR',
    aggfunc='count',
)

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
NERC.REGION,,,,,,,
ECAR,2.0,NaN,1.0,NaN,NaN,28.0,3.0
FRCC,4.0,NaN,2.0,NaN,3.0,26.0,8.0
"FRCC, SERC",NaN,NaN,NaN,NaN,NaN,NaN,1.0
HECO,NaN,NaN,NaN,NaN,NaN,2.0,1.0
HI,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...
RFC,8.0,5.0,106.0,5.0,3.0,279.0,12.0
SERC,7.0,2.0,30.0,1.0,13.0,130.0,18.0
SPP,1.0,1.0,8.0,2.0,16.0,36.0,2.0


## Step 3: Assessment of Missingness

In [583]:
#Calculate observed statistic
def tvd(dist1, dist2):
    return np.abs(dist1 - dist2).sum() / 2
temp = df.assign(missing=df['OUTAGE.RESTORATION'].isna()).rename(columns={'MONTH' : 'Month'})
dist = temp.pivot_table(
    index='Month',
    columns='missing',
    aggfunc='size'
)
obs = np.abs((dist / dist.sum()).diff(axis=1)[dist.columns[-1]]).sum() / 2
obs

np.float64(0.14349520687596456)

In [566]:
#Run permutation test
tvds = []
for i in range(1000):
    temp['missing'] = np.random.permutation(temp['missing'])
    test_dist = temp.pivot_table(
    index='Month',
    columns='missing',
    aggfunc='size'
    )
    tvd = np.abs((test_dist / test_dist.sum()).diff(axis=1)[test_dist.columns[-1]]).sum() / 2
    tvds.append(tvd)
(tvds >= obs).mean()

np.float64(0.219)

In [567]:
dist = dist / dist.sum()
fig5 = dist.plot(kind='barh', title='Month by Missingness of Outage Restoration', barmode='group')
fig5

In [568]:
fig6 = px.histogram(pd.DataFrame(tvds), x=0, histnorm='probability', labels={'0' : 'TVD'}, title='Empirical Distribution of the TVD')
fig6.add_vline(x=obs, line_width=1.5, line_color='red')
fig6

In [575]:
#Calculate observed statistic
temp = df.assign(missing=df['OUTAGE.RESTORATION'].isna()).rename(columns={'CLIMATE.REGION' : 'Climate Region'})
dist = temp.pivot_table(
    index='Climate Region',
    columns='missing',
    aggfunc='size'
)
obs = np.abs((dist / dist.sum()).diff(axis=1)[dist.columns[-1]]).sum() / 2
obs

np.float64(0.2908128946193284)

In [576]:
#Run permutation test
tvds = []
for i in range(1000):
    temp['missing'] = np.random.permutation(temp['missing'])
    test_dist = temp.pivot_table(
    index='Climate Region',
    columns='missing',
    aggfunc='size'
    )
    tvd = np.abs((test_dist / test_dist.sum()).diff(axis=1)[test_dist.columns[-1]]).sum() / 2
    tvds.append(tvd)
(tvds >= obs).mean()

np.float64(0.0)

In [577]:
dist = dist / dist.sum()
fig7 = dist.plot(kind='barh', title='Climate Region by Missingness of Outage Restoration', barmode='group')
fig7

In [578]:
fig8 = px.histogram(pd.DataFrame(tvds), x=0, histnorm='probability', labels={'0' : 'TVD'}, title='Empirical Distribution of the TVD')
fig8.add_vline(x=obs, line_width=1.5, line_color='red')
fig8

## Step 4: Hypothesis Testing

In [282]:
#H0: The distribution of Cause Category for outages occur at day time is the same as the distribution for outages occur at night time.
#Ha: The distribution of Cause Category for outages occur at day time is different from the distribution for outages occur at night time.

In [593]:
#Calculate observed statistic
temp = df[['DAY', 'CAUSE.CATEGORY']].rename(columns={'CAUSE.CATEGORY' : 'Cause Category', 'DAY' : 'Day'})
dist = temp.pivot_table(
    index='Cause Category',
    columns='Day',
    aggfunc='size'
)
obs = np.abs((dist / dist.sum()).diff(axis=1)[dist.columns[-1]]).sum() / 2
obs

np.float64(0.24867575288656116)

In [595]:
#Run permutation test
tvds = []
for i in range(1000):
    temp['Day'] = np.random.permutation(temp['Day'])
    test_dist = temp.pivot_table(
    index='Cause Category',
    columns='Day',
    aggfunc='size'
    )
    tvd = np.abs((test_dist / test_dist.sum()).diff(axis=1)[test_dist.columns[-1]]).sum() / 2
    tvds.append(tvd)
(tvds >= obs).mean()

np.float64(0.0)

In [596]:
dist = dist / dist.sum()
fig9 = dist.plot(kind='barh', title='Cause Category by Day/Night', barmode='group')
fig9

In [597]:
fig10 = px.histogram(pd.DataFrame(tvds), x=0, histnorm='probability', labels={'0' : 'TVD'}, title='Empirical Distribution of the TVD')
fig10.add_vline(x=obs, line_width=1.5, line_color='red')

## Step 5: Framing a Prediction Problem

In [598]:
#Classify CAUSE.CATEGORY into two major categories.
#Natural/Systemic Causes(Encoded 1): Outages caused by natural events or systemic issues within the infrastructure, usually due to force majeure.
#Includes severe weather, system operability disruption, equipment failure
#Human-Induced Causes(Encoded 0): Outages caused by human actions or decisions, whether intentional or unintentional.
#Includes intentional attack, public appeal, fuel supply emergency, islanding
#My model will predict which of the two categories the cause of a power outage will be.

## Step 6: Baseline Model

In [603]:
df['CAUSE'] = df['CAUSE.CATEGORY'].isin(['severe weather', 'system operability disruption', 'equipment failure']).astype(int)
df['CAUSE']

0       1
1       0
2       1
3       1
4       1
       ..
1520    0
1521    1
1522    0
1523    0
1524    0
Name: CAUSE, Length: 1525, dtype: int64

In [604]:
#Make and split the train/test set
y = df['CAUSE']
X_train, X_test, y_train, y_test = train_test_split(df, y)
X_train_0 = X_train[['NERC.REGION', 'TOTAL.CUSTOMERS', 'MONTH', 'HOUR']]
X_test_0 = X_test[['NERC.REGION', 'TOTAL.CUSTOMERS', 'MONTH', 'HOUR']]

#Build ColumnTransformer with 4 features
preproc = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['NERC.REGION']),
        ('std', StandardScaler(), ['TOTAL.CUSTOMERS'])
    ],
    remainder='passthrough',
    force_int_remainder_cols=False
)

#Build and train DecisionTreeClassifier
clf = Pipeline([
    ('preprocessor', preproc),
    ('clf', DecisionTreeClassifier(
        max_depth=5,
        criterion='entropy'
    ))
])

clf.fit(X_train_0, y_train)
clf.score(X_test, y_test)

0.725130890052356

## Step 7: Final Model

In [469]:
#Now we will need these features
X_train_1 = X_train[['NERC.REGION', 'TOTAL.CUSTOMERS', 'MONTH', 'HOUR', 'POPPCT_URBAN', 'YEAR']]
X_test_1 = X_test[['NERC.REGION', 'TOTAL.CUSTOMERS', 'MONTH', 'HOUR', 'POPPCT_URBAN', 'YEAR']]

#Transfrom cyclic features into sine
def sine_m(s):
    return np.sin(2 * np.pi * s / 12)

def sine_h(s):
    return np.sin(2 * np.pi * s / 24)

#Build ColumnTransformer
preproc = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['NERC.REGION']),
        ('month', FunctionTransformer(sine_m), ['MONTH']),
        ('hour', FunctionTransformer(sine_h), ['HOUR']),
        ('std', StandardScaler(), ['TOTAL.CUSTOMERS', 'POPPCT_URBAN'])
    ],
    remainder='passthrough',
    force_int_remainder_cols=False
)

#Build and train RandomForestClassifier
clf = Pipeline([
    ('preprocessor', preproc),
    ('clf', RandomForestClassifier(
        max_depth=5,
        criterion='entropy'
    ))
])

clf.fit(X_train_1, y_train)

#Use GridSearchCV to find the best hyperparameters for the RandomForestClassifier
hyperparameters = {
    'clf__max_depth': np.arange(1, 20),
    'clf__min_samples_split': [2, 5, 10, 20, 50, 100, 200],
    'clf__criterion': ['gini', 'entropy']
}
grids = GridSearchCV(
    clf,
    n_jobs=-1,
    param_grid=hyperparameters,
    return_train_score=True
)
grids.fit(X_train_1, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(force_int_remainder_cols=False,
                                                          remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['NERC.REGION']),
                                                                        ('month',
                                                                         FunctionTransformer(func=<function sine_m at 0x00000185DD029A80>),
                                                                         ['MONTH']),
                                                                        ('hour',
                                                                         FunctionTransformer(func=<function sine_h at 0x00000185DD02AAC0>),
                                                                         ['HOUR']),
                                                                        ('std',
                                                                         StandardScaler(),
                                                                         ['TOTAL.CUSTOMERS',
                                                                          'POPPCT_URBAN'])])),
                                       ('clf',
                                        RandomForestClassifier(criterion='entropy',
                                                               max_depth=5))]),
             n_jobs=-1,
             param_grid={'clf__criterion': ['gini', 'entropy'],
                         'clf__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                         'clf__min_samples_split': [2, 5, 10, 20, 50, 100,
                                                    200]},
             return_train_score=True)

In [470]:
grids.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': np.int64(14),
 'clf__min_samples_split': 5}

In [471]:
#Build RandomForestClassifier with the hyperparameters I found
final_clf = Pipeline([
    ('preprocessor', preproc),
    ('clf', RandomForestClassifier(
        max_depth=13,
        min_samples_split=5,
        criterion='entropy'
    ))
])
final_clf.fit(X_train_1, y_train)
final_clf.score(X_test_1, y_test)

0.8141361256544503

## Step 8: Fairness Analysis

In [ ]:
#H0: The classifier's accuracy is the same for years before and after 2010
#Ha: The classifier's accuracy is higher for years before 2010

0.8141361256544503

In [ ]:
#Add the prediction of model to the dataframe
y_pred = final_clf.predict(X_test)
metrics.accuracy_score(y_pred, y_test)
results = X_test_1
results['Before 2010'] = results['YEAR'] < 2010
results['prediction'] = y_pred
results['Cause'] = y_test

In [ ]:
compute_accuracy = lambda x: metrics.accuracy_score(x['Cause'], x['prediction'])

#Calculate observed statistic
obs = results.groupby('Before 2010')[['Cause', 'prediction']].apply(compute_accuracy).diff().iloc[-1]
obs

np.float64(0.13215728201473032)

In [505]:
#Run permutation test
diffs = []
temp = results[['Before 2010', 'prediction', 'Cause']]
for i in range(1000):
    temp['Before 2010'] = np.random.permutation(temp['Before 2010'])
    diff = temp.groupby('Before 2010')[['Cause', 'prediction']].apply(compute_accuracy).diff().iloc[-1]
    diffs.append(diff)
(diffs >= obs).mean()

np.float64(0.0)

In [512]:
fig11 = pd.Series(diffs).plot(kind='hist', histnorm='probability', nbins=20, title='Difference in Accuracy(Before 2010 - After 2010)')
fig11.add_vline(x=obs, line_color='red', line_width=1.5)
fig11